In [4]:
import json
import argparse
import dotenv

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

questions = "datasets/spider_data/dev.json"

system_prompt = """You are an expert SQL assistant specialized in converting natural language queries into accurate SQL statements. You:
1. Understand database schemas and relationships
2. Generate standard SQL queries that follow best practices
3. Consider edge cases and data validation
4. Can handle complex joins, aggregations, and nested queries
5. Provide explanations for the generated SQL when needed

When given a question, you will convert it to a valid SQL query based on the provided database schema."""


# Load questions from JSON file
with open(questions, 'r') as f:
    questions = json.load(f)

In [2]:
questions[0]

{'db_id': 'concert_singer',
 'query': 'SELECT count(*) FROM singer',
 'query_toks': ['SELECT', 'count', '(', '*', ')', 'FROM', 'singer'],
 'query_toks_no_value': ['select', 'count', '(', '*', ')', 'from', 'singer'],
 'question': 'How many singers do we have?',
 'question_toks': ['How', 'many', 'singers', 'do', 'we', 'have', '?'],
 'sql': {'from': {'table_units': [['table_unit', 1]], 'conds': []},
  'select': [False, [[3, [0, [0, 0, False], None]]]],
  'where': [],
  'groupBy': [],
  'having': [],
  'orderBy': [],
  'limit': None,
  'intersect': None,
  'union': None,
  'except': None}}